In [6]:
import os
import wandb
import torch
import random
import numpy as np
import pandas as pd
from rich import print
from pathlib import Path
from tqdm.auto import tqdm
from dotenv import load_dotenv
from collections import defaultdict
from huggingface_hub import login
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
import transformers
from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
sns.set_theme(context='notebook', style='dark')
tqdm.pandas()

In [8]:
seed = 42

random.seed(seed)

np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

transformers.set_seed(seed)

In [9]:
dataset_path = Path('./sentences.txt')
tokenizer_path = Path('./models/nano-gpt-tokenizer/')
model_ckpt = 'openai-community/gpt2'
model_name = 'ArabMindGPT'
model_path = Path(f'./{model_name}')

In [10]:
PROJECT_NAME = 'ArabMindGPT'
JOB_TYPE = 'LM-Modeling'
TAGS = ['MOdeling', 'Transformers', 'GPT2', 'Language-Modeling', 'Arabic']
NOTES = 'LM Training on Arabic Data using GPT2 Model Architecture'
RUN_NAME = 'Dummy-data-and-model-LM-Moldeing'
config = defaultdict(dict)

In [11]:
load_dotenv()
wandb.login()
login(token=os.getenv('HF_TOKEN'),
      add_to_git_credential=True, write_permission=True)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mazin-sherif100 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)


In [12]:
# load the dataset
with open(dataset_path, 'r') as f:
    dataset = f.readlines()[:100_000]

In [13]:
train, valid = train_test_split(dataset, train_size=0.8, random_state=seed)
valid, test = train_test_split(valid, train_size=0.5, random_state=seed)

In [14]:
dataset = DatasetDict()
dataset['train'] = Dataset.from_dict({'text': train})
dataset['valid'] = Dataset.from_dict({'text': valid})
dataset['test'] = Dataset.from_dict({'text': test})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 80000
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10000
    })
})

In [15]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
print(tokenizer)

GPT2TokenizerFast(name_or_path='models/nano-gpt-tokenizer', vocab_size=4096, model_max_length=128, is_fast=True, 
padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': 
'<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
        0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, 
special=True),
}
)

In [16]:
def tokenize(batch):
    outputs = tokenizer(
        batch['text'],
        truncation=True,
        return_overflowing_tokens=True,
    )
    return {'input_ids': outputs['input_ids']}


tokenized_dataset = dataset.map(
    tokenize, batched=True, remove_columns=dataset['train'].column_names
)
print(tokenized_dataset)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 80065
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 10008
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 10005
    })
})

In [17]:
model_config = AutoConfig.from_pretrained(
    model_ckpt, vocab_size=tokenizer.vocab_size,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    n_positions=tokenizer.model_max_length,
    n_ctx=tokenizer.model_max_length,
    n_embd=128, n_head=2, n_layer=2,
)
print(model_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

GPT2Config {
  "_name_or_path": "openai-community/gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 128,
  "n_embd": 128,
  "n_head": 2,
  "n_inner": null,
  "n_layer": 2,
  "n_positions": 128,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.47.1",
  "use_cache": true,
  "vocab_size": 4096
}

In [18]:
model = AutoModelForCausalLM.from_config(model_config)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(4096, 128)
    (wpe): Embedding(128, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPT2Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=384, nx=128)
          (c_proj): Conv1D(nf=128, nx=128)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=512, nx=128)
          (c_proj): Conv1D(nf=128, nx=512)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=128, out_features=4096, bias=False)
)

In [19]:
print('Size in MBs:', model.get_memory_footprint() / 1_000_000)

Size in MBs: 3.782664

In [20]:
print('Num Params:', model.num_parameters() / 1_000_000, 'M')

Num Params: 0.937472 M

In [21]:
num_epochs = 5
batch_size = 16 * 32
total_steps = len(tokenized_dataset['train']) * num_epochs // batch_size
total_steps

781

In [22]:
training_args = TrainingArguments(
    output_dir=model_path,
    run_name=RUN_NAME,
    report_to='wandb',
    save_strategy='no',
    eval_strategy='steps',
    gradient_accumulation_steps=32,
    overwrite_output_dir=True,
    data_seed=seed, seed=seed,
    learning_rate=1e-3,
    weight_decay=0.001,
    warmup_ratio=0.0,
    eval_steps=500,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

In [23]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['valid'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-24-02a9a709293e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
run = wandb.init(project=PROJECT_NAME, job_type=JOB_TYPE,
                 name=RUN_NAME, notes=NOTES, tags=TAGS, config=config)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [26]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=20, training_loss=246.9587890625, metrics={'train_runtime': 8010.9616, 'train_samples_per_second': 49.972, 'train_steps_per_second': 0.002, 'total_flos': 88035203481600.0, 'train_loss': 246.9587890625, 'epoch': 4.8152866242038215})

In [27]:
trainer.evaluate(tokenized_dataset['test'], metric_key_prefix='test')

{'test_loss': 7.354394912719727,
 'test_runtime': 74.7658,
 'test_samples_per_second': 133.818,
 'test_steps_per_second': 0.268,
 'epoch': 4.8152866242038215}

In [28]:
wandb.finish()

test/loss,▁
test/runtime,▁
test/samples_per_second,▁
test/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
test/loss,7.35439
test/runtime,74.7658
test/samples_per_second,133.818
test/steps_per_second,0.268
total_flos,88035203481600.0


In [29]:
trainer.save_model(model_path)

In [30]:
_ = trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/3.75M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]